#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load embedding model into ollama:
    1. Attach ollama container shell
    2. Run command in container `ollama pull pull bge-m3` or send an api call `requests.post("http://localhost:11434/api/pull", json = {"name": "bge-m3",  "stream": False}` via the requests library to download embedding model.
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Ollama REST API](https://www.postman.com/postman-student-programs/ollama-api/documentation/suc47x8/ollama-rest-api)
- [Langchain Ollama Embeddings](https://python.langchain.com/docs/integrations/text_embedding/ollama/)
- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [43]:
import requests

In [ ]:
# Download embedding model into ollama container
requests.post("http://localhost:11434/api/pull", json = {"name": "bge-m3",  "stream": False})

In [ ]:
# List all available models in the ollama container
response = requests.get("http://localhost:11434/api/ps")
response.json()

{'models': [{'name': 'bge-m3:latest',
   'model': 'bge-m3:latest',
   'size': 1189142528,
   'digest': '7907646426070047a77226ac3e684fbbe8410524f7b4a74d02837e43f2146bab',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'bert',
    'families': ['bert'],
    'parameter_size': '566.70M',
    'quantization_level': 'F16'},
   'expires_at': '2025-02-10T13:42:02.900213833Z',
   'size_vram': 0},
  {'name': 'llama3.2:1B',
   'model': 'llama3.2:1B',
   'size': 2233526272,
   'digest': 'baf6a787fdffd633537aa2eb51cfd54cb93ff08e28040095462bb63daf552878',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '1.2B',
    'quantization_level': 'Q8_0'},
   'expires_at': '2025-02-10T13:06:59.841731263Z',
   'size_vram': 0}]}

In [1]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model="bge-m3",
)

In [20]:
text = "This is a test document."

query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 1024
[-0.050017744, 0.021129876, -0.057069883, -0.0055166543, -0.0062899087, -0.02841217, 0.045161195, 0.042706154, -0.00025403852, 0.013057328]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [26]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
pages_chunked = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter())

print(pages_chunked[0])

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Apply cleaning while preserving the Document structure
pages_chunked_cleaned = [
    Document(page_content=clean_text(doc.page_content), metadata=doc.metadata)
    for doc in pages_chunked
]

print(pages_chunked_cleaned[0])

page_content='Aurélien Géron
Hands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
SECOND EDITION
Boston Farnham Sebastopol TokyoBeijing Boston Farnham Sebastopol TokyoBeijing' metadata={'source': './AI_Book.pdf', 'page': 2, 'page_label': '1'}
page_content='Aurlien Gron
Hands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
SECOND EDITION
Boston Farnham Sebastopol TokyoBeijing Boston Farnham Sebastopol TokyoBeijing' metadata={'source': './AI_Book.pdf', 'page': 2, 'page_label': '1'}


In [27]:
print(pages_chunked_cleaned[1])

page_content='978-1-492-03264-9
[LSI]
Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurlien Gron
Copyright  2019 Aurlien Gron. All rights reserved.
Printed in the United States of America.
Published by OReilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
OReilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles (http://oreilly.com). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com.
Editor: Nicole Tache
Interior Designer: David Futato
Cover Designer: Karen Montgomery
Illustrator: Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649 for release details.
The OReilly logo is a registere

In [29]:
print(f"Number of text chunks: {len(pages_chunked_cleaned)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [55]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection_name = "ai_model_book"

collection = client.get_or_create_collection(collection_name)

vector_db_from_client = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_model,
)

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned))]

vector_db_from_client.add_documents(documents=pages_chunked_cleaned, ids=uuids) # that can take a while...

['35ee54f3-bb96-4901-80a2-ac56a06f1a4b',
 'ff1eeec0-96cd-403a-8699-ba5d08e911cc',
 'dbb5ab22-8082-4557-a85d-d23e3389fe8b',
 '06f0cdd6-947a-49e6-be4f-8f10bdccf987',
 'e06ff6fa-05b6-4080-a730-b94c0963fe55',
 'f581d261-af89-42d6-bf9b-aed572c7e772',
 'f5e8a360-0ebb-41a7-ae87-a6bc97be825d',
 '4983d824-ed89-44e9-9dfb-8b9426bcfd39',
 '5bf9604c-f7d0-4e5c-9135-0059181b36e0',
 'd24c4d56-c0be-4162-a203-f14766bccdaa',
 'b62cf006-621a-4271-8acb-4840d7ab6f53',
 'cb17c536-f854-46b4-a3e7-982c9c540b77',
 'fed7579a-e989-4229-8798-21c5edc81043',
 '1b9f4f6f-4b32-43b4-bd52-f9895b9f26f4',
 'fd6dea2d-9535-441e-84cc-4c241255ad5d',
 'f72b6778-4a44-45f3-b417-f5e86dc862c3',
 'a7f1d7a2-03ce-4eba-98d0-7cb8676909e1',
 '51b54f2b-3df5-4fcc-8e68-9cf6bfb04491',
 '432569f0-2ffd-41c3-91fa-e36c75ea13c0',
 '7efd826d-938c-40bc-afdb-266d51e2ab89',
 '70db087a-f1c4-48b4-a3bf-46e0d8a30257',
 '6f145883-fdb9-4ed1-9d65-018d9b515b31',
 '3c9f3ecc-7e1f-4020-8bcb-fb4d727eaddf',
 '5f3b4a0f-8700-4707-9dbe-5a7a2334882b',
 '3615b939-e385-

In [ ]:
# Check the number of collections
client.count_collections()

1

In [ ]:
# Deletion of a collection
# client.delete_collection("ai_model_book")

In [ ]:
# Get and check the content of the collection
collection.get(include=["documents", "metadatas", "embeddings"])

{'ids': ['35ee54f3-bb96-4901-80a2-ac56a06f1a4b',
  'ff1eeec0-96cd-403a-8699-ba5d08e911cc',
  'dbb5ab22-8082-4557-a85d-d23e3389fe8b',
  '06f0cdd6-947a-49e6-be4f-8f10bdccf987',
  'e06ff6fa-05b6-4080-a730-b94c0963fe55',
  'f581d261-af89-42d6-bf9b-aed572c7e772',
  'f5e8a360-0ebb-41a7-ae87-a6bc97be825d',
  '4983d824-ed89-44e9-9dfb-8b9426bcfd39',
  '5bf9604c-f7d0-4e5c-9135-0059181b36e0',
  'd24c4d56-c0be-4162-a203-f14766bccdaa',
  'b62cf006-621a-4271-8acb-4840d7ab6f53',
  'cb17c536-f854-46b4-a3e7-982c9c540b77',
  'fed7579a-e989-4229-8798-21c5edc81043',
  '1b9f4f6f-4b32-43b4-bd52-f9895b9f26f4',
  'fd6dea2d-9535-441e-84cc-4c241255ad5d',
  'f72b6778-4a44-45f3-b417-f5e86dc862c3',
  'a7f1d7a2-03ce-4eba-98d0-7cb8676909e1',
  '51b54f2b-3df5-4fcc-8e68-9cf6bfb04491',
  '432569f0-2ffd-41c3-91fa-e36c75ea13c0',
  '7efd826d-938c-40bc-afdb-266d51e2ab89',
  '70db087a-f1c4-48b4-a3bf-46e0d8a30257',
  '6f145883-fdb9-4ed1-9d65-018d9b515b31',
  '3c9f3ecc-7e1f-4020-8bcb-fb4d727eaddf',
  '5f3b4a0f-8700-4707-9dbe-

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [41]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(
    search_query,
    k=3
)
for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
 Whether or not they are trained with human supervision (supervised, unsuper
vised, semisupervised, and Reinforcement Learning)
 Whether or not they can learn incrementally on the fly (online versus batch
learning)
 Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Lets look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems can